# EEP/IAS C118 - Spring 2024- Section 2

## Function of the Week: group_by()

This week, we will work review usage of the function `group_by()`, which was also covered in R Bootcamp part 1.

## Preamble & Load Dataset

Begin by importing the "SLEEP75.DTA" dataset. Note that reading Stata data files (.dta or .DTA) requires the use of the `haven` package, specifically the `read_dta()` function. Let us also call in the `dplyr` function, which will come in handy later on.

In [1]:
# Load "haven" and "dplyr" package
library(haven) # DELETE for Blank
library(dplyr) # DELETE for Blank


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
# Load in "sleep75.dta" and save it as data-frame object "sleep75"
sleep75 <- read_dta("sleep75.dta") # DELETE for Blank

In [3]:
# Use head() function to explore the dataframe
head(sleep75) # DELETE for Blank

age,black,case,clerical,construc,educ,earns74,gdhlth,inlf,leis1,⋯,spwrk75,totwrk,union,worknrm,workscnd,exper,yngkid,yrsmarr,hrwage,agesq
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
32,0,1,0,0,12,0,0,1,3529,⋯,0,3438,0,3438,0,14,0,13,7.070004,1024
31,0,2,0,0,14,9500,1,1,2140,⋯,0,5020,0,5020,0,11,0,0,1.429999,961
44,0,3,0,0,17,42500,1,1,4595,⋯,1,2815,0,2815,0,21,0,0,20.529997,1936
30,0,4,0,0,12,42500,1,1,3211,⋯,1,3786,0,3786,0,12,0,12,9.619998,900
64,0,5,0,0,14,2500,1,1,4052,⋯,1,2580,0,2580,0,44,0,33,2.750000,4096
41,0,6,0,0,12,0,1,1,4812,⋯,0,1205,0,0,1205,23,0,23,19.249998,1681


## Summary stats by groups

Sometimes we may want to group our data by values of certain variables.

For example, we may want to know if the levels of income in our sample differ systematically by gender.

Below, we cover three methods for doing so.

Note: The relevant varibles in the dataframe are: `earns74` for income and `male` for gender.

### Method 1: Using `filter()`

If we want summary statistics from our _sleep75_ data split on the variable _male_, we could use `filter()` to subset the dataset by gender to create two separate datasets and run each through `summarise()`.

Note: `filter()` function comes from `dplyr` package.

In [4]:
# Create a new smaller dataframe with subset of observations from "sleep75" with only female observations
sleep75_male <- filter(sleep75, male == 0) # DELETE for Blank

# Create a new smaller dataframe with subset of observations from "sleep75" with only male observations
sleep75_female <- filter(sleep75, male == 1) # DELETE for Blank

In [5]:
# Use summarise() to find mean income for females only
summarise(sleep75_female, 
          "Count of Observations" = n(), 
          "Average Income" = mean(earns74)) # DELETE for Blank

Count of Observations,Average Income
<int>,<dbl>
400,10956.25


In [6]:
# Use summarise() to find mean income for males only
summarise(sleep75_male, 
          "Count of Observations" = n(), 
          "Average Income" = mean(earns74)) # DELETE for Blank

Count of Observations,Average Income
<int>,<dbl>
306,8214.052


However, this requires storing multiple objects in memory and running identical summarise() codes twice.

### Method 2: Using `group_by()`

Instead, we can perform the same task more cleanly with group by:

1 Group the data by the variable male: `sleep75 <- group_by(sleep75, male)`

2 Produce summary statistics using summarise(): `summarise(sleep75, "Count of Observations" = n(), "Average Income" = mean(earns74))`

This code produces a table of summary statistics, with one row for females (male = 0) and one row for males (male = 1), as opposed to the default of one row for the entire dataset.

In [7]:
sleep75 <- group_by(sleep75, male)

summarise(sleep75, "Count of Observations" = n(), "Average Income" = mean(earns74))

male,Count of Observations,Average Income
<dbl>,<int>,<dbl>
0,306,8214.052
1,400,10956.250


**First column**: the variable we grouped by

**Other columns**: our custom summary statistics

**First row**: summary stats for women and other genders (male = 0)

**Second row**: summary stats for men (male = 1)

Let's say we want to further create variables based on group membership. We can do that too! If we want a variable equal to this mean of income conditional on gender, we can add it to our grouped data using `mutate()`, again from `dplyr` package:

`sleep75 <- mutate(sleep75, inc_avg = mean(earns74))`

In [8]:
sleep75 <- mutate(sleep75, inc_avg = mean(earns74))

In [9]:
# Take a look at your dataset. It added a new variable named 'inc_avg', and its values are 10956.2 for males and 8214 for females.
head(sleep75)

age,black,case,clerical,construc,educ,earns74,gdhlth,inlf,leis1,⋯,totwrk,union,worknrm,workscnd,exper,yngkid,yrsmarr,hrwage,agesq,inc_avg
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
32,0,1,0,0,12,0,0,1,3529,⋯,3438,0,3438,0,14,0,13,7.070004,1024,10956.250
31,0,2,0,0,14,9500,1,1,2140,⋯,5020,0,5020,0,11,0,0,1.429999,961,10956.250
44,0,3,0,0,17,42500,1,1,4595,⋯,2815,0,2815,0,21,0,0,20.529997,1936,10956.250
30,0,4,0,0,12,42500,1,1,3211,⋯,3786,0,3786,0,12,0,12,9.619998,900,8214.052
64,0,5,0,0,14,2500,1,1,4052,⋯,2580,0,2580,0,44,0,33,2.750000,4096,10956.250
41,0,6,0,0,12,0,1,1,4812,⋯,1205,0,0,1205,23,0,23,19.249998,1681,10956.250


Note that all functions like mean(), sd(), min(), quantile(), etc.
behave this way with grouped data

Grouped output from summarise() will have as many rows as values of the grouping variable – we would have had 15 rows if we grouped
on years of education!

When done with grouped transformations, make sure to use ungroup() on your data to restore the functions to their normal behavior:

`sleep75 <- ungroup(sleep75)`

You can group by multiple variables, just separate them by commas (i.e. age then education level for all duplicated ages)

We could also want to create a new variable equal to the mean of income conditional on race

With `group_by()` in tidyverse package we can do both of these!